In [28]:
import pandas as pd
import whylogs
from whylogs.viz import NotebookProfileVisualizer
reference = pd.read_csv(r'/Users/s/Desktop/mlops/network_Security_end2end/artifact/data/transformed/train_transformed.csv')
current = pd.read_csv(r'/Users/s/Desktop/mlops/network_Security_end2end/artifact/data/transformed/test_transformed.csv')
reference_profile = whylogs.log(reference)
current_profile = whylogs.log(current)
reference_profile_view = reference_profile.view()
current_profile_view = current_profile.view()
visualization = NotebookProfileVisualizer()
visualization.set_profiles(target_profile_view=current_profile_view, reference_profile_view=reference_profile_view)
visualization


In [35]:
import os
%pip install 'whylogs[viz]'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for pybars3: filename=pybars3-0.9.7-py3-none-any.whl size=14131 sha256=fad6915f1f7112397bb34665124ae877073cf096ff1784536069eac2195e7dc0
  Stored in directory: /Users/s/Library/Caches/pip/wheels/5b/79/29/fb952aaf1fe2d19c7bfbe7c7ba96bf135caa44cb6f0fe372f6
  Created wheel for PyMeta3: filename=PyMeta3-0.5.1-py3-none-any.whl size=16490 sha256=c1717f809db7bf02d609c77b9f249442326ed19d6d420bd53c8881da6b20af4d
  Stored in directory: /Users/s/Library/Caches/pip/wheels/8f/66/2c/fbe2a7aae963eb2d885051fe159fa8bf41cd198b14b5fbad59
Successfully built pybars3 PyMeta3
  Attempting uninstall: Pillow
    Found existing installation: pillo

In [36]:
os.getcwd()
visualization.write(
    rendered_html=visualization.profile_summary(),
    html_file_name=os.getcwd() + "/example",
)

In [37]:
from whylogs.viz.drift.column_drift_algorithms import calculate_drift_scores

scores = calculate_drift_scores(target_view=current_profile_view, reference_view=reference_profile_view, with_thresholds = True)

print(scores)

{'Max Packet Length': {'algorithm': 'ks', 'pvalue': np.float64(0.2281434065906851), 'statistic': 0.0023367680069312224, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}, 'Packet Length Variance': {'algorithm': 'ks', 'pvalue': np.float64(0.06331245344216374), 'statistic': 0.0029482164272708333, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'POSSIBLE_DRIFT'}, 'Packet Length Std': {'algorithm': 'ks', 'pvalue': np.float64(0.22827289659823125), 'statistic': 0.0023364609073396325, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}, 'Destination Port': {'algorithm': 'ks', 'pvalue': np.float64(0.5733802222871651), 'statistic': 0.0017547498201294642, 'thresholds': {'NO_DRIFT': (0.15, 1), 'POSSIBLE_DRIFT': (0.05, 0.15), 'DRIFT': (0, 0.05)}, 'drift_category': 'NO_DRIFT'}, 'Avg Bwd Segment Size':

In [ ]:
visualization.summary_drift_report()